# APO Lab 3 

## Posterize

Redukcja poziomów szarości przez posteryzację. Polega na równomiernym rozłożeniu zakresów jasności które są ujednolicane do wspólnej wartości. Na przykład dla 2 poziomów posteryzacja odpowiada binaryzacji z progiem 127. 

Realizacja tej metody przez UOP przyjmuje postać wykresu schodkowego (jak na rysunku poniżej).

![alt text](https://github.com/knave88/APO/raw/main/posterize.png)


In [ ]:
myPosterizeBinsNum = 8


print('my number of Bins: '+str(myPosterizeBinsNum)) 
print('my Bin length: '+str(np.round(255/myPosterizeBinsNum))) 
print('my Posterization Bins: '+str(np.arange(0,255,np.round(255/myPosterizeBinsNum))))

In [ ]:
# calc size of binning
myBins = np.arange(0,255,np.round(255/myPosterizeBinsNum))

#init output image
img_pstrz = np.zeros_like(img_gray)
# loop through image
for h in range(img_gray.shape[0]):
  for w in range(img_gray.shape[1]):
    current_pixel = img_gray[h,w]

    # loop through bins
    for bin in range(myPosterizeBinsNum):
      #print(myBins[bin])
      if (current_pixel>myBins[bin]): img_pstrz[h,w]=myBins[bin] # if inside bin assign value

    if (current_pixel>myBins[-1]): img_pstrz[h,w]=255 # last bin -> fill with max value

# display output image
#cv2_imshow(img_pstrz)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_pstrz, cv2.COLOR_BGR2RGB))

## Posterize with LUT

In [ ]:
# init LUT
bin_length = np.round(255/myPosterizeBinsNum).astype(int)
lut_pstrz = []

# popualte LUT
for bin in range(myPosterizeBinsNum):
  temp = np.ones(bin_length,)*myBins[bin] #temp vector
  lut_pstrz = np.hstack((lut_pstrz, temp)) #concatenate
lut_pstrz = np.hstack((lut_pstrz, np.ones(bin_length,)*255)) #last bin 

lut_pstrz.shape # check LUT size

In [ ]:
# show LUT
lut_pstrz

In [ ]:
# apply LUT
img_pstrz2 = lut_pstrz[img_gray].astype('uint8')

#cv2_imshow(img_pstrz2)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_pstrz2, cv2.COLOR_BGR2RGB))

Podsumowując, do zaimplementowania operacji punktowych często jest konieczne pobieranie od użytkownika dodatkowych wartości parametrów. Można to rozwiązać jako pole do wpisywania wartości lub suwak ograniczający możliwość wyboru wartości. Operacje punktowe mają swoje odniesienie w UOP a co za tym idzie można w prosty sposób rozwiązać stosując tablicę LUT. 

# Operacje sąsiedztwa

In [ ]:
#from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from matplotlib import pyplot as plt

!WAŻNE! Aby poniższy plik działał na Google Colab, należy wgrać plik 'lena_gray.bmp' oraz 'testing.bmp'. Z panelu po lewej stronie wybieramy ikonę folderu ('Files') a następnie 'Upload' i wybieramy zdjęcie (wcześniej ściągnięte na dysk twardy z UBI).

In [ ]:
# Wczytanie obrazu pierwotnego
img = cv2.imread('lena_gray.bmp', cv2.IMREAD_GRAYSCALE)

#cv2_imshow(img)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

## 1. a) blur

In [ ]:
# ustawienie rozmiaru maski - okna przekształcenia (kernel) 
kernel_size = (5,5)
# standardowe wygładzanie liniowe funkcją blur z biblioteki OpenCV przy użyciu powyższego rozmiaru maski
blured_img = cv2.blur(img,kernel_size)

#cv2_imshow(blured_img)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(blured_img, cv2.COLOR_BGR2RGB))

Funkcja blur używa maski przekształcenia w postaci:

![alt text](https://github.com/knave88/APO/raw/main/opencv_blur.png)

czyli tzw. normalized box filter.
Dla przypomnienia im większe okno tym większe rozmycie.

In [ ]:
# funkcja ta może przyjąć jako dodatkowy parametr określenie metody ekstrapolacji wartości brzegowych obrazu
blured_img = cv2.blur(img,(5,5), borderType = cv2.BORDER_REPLICATE)

# pozostałe opcje do zaimplementowania:
# 1.	pozostawienie wartości pikseli brzegowych bez zmian (cv2.BORDER_ISOLATED), 
# 2.	powielenie wartości pikseli brzegowych przez odbicie lustrzane (cv2.BORDER_REFLECT), 
# 3.	powielenie wartości pikseli brzegowych przez powielenie skrajnego piksela (cv2.BORDER_REPLICATE)

## 1. a) gaussianBlur

Funkcja gaussianBlur, zgodnie z nazwą, używa filtra gaussowskiego do wygładzenia obrazu.
Parametry wejściowe które należy ustawić:

* rozmiar filtra
* odchylenie standardowe w kierunku X
* odchylenie standardowe w kierunku Y (jeśli nie podamy wartości przyjmowana jest wartość z kierunku X)
* metoda ekstrapolacji pikseli brzegowych




In [ ]:
kernel_size = (5,5) # rozmiar filtra
sigmaX = 0 # odch. std. w kiernku X; !Uwaga jeśli podamy wartość 0, parametr jest liczony automatycznie na podstawie rozmiaru kernela
borderType = cv2.BORDER_REPLICATE # metoda ekstrapolacji wartości brzegowych
                                  # alternatywne opcje: BORDER_REPLICATE BORDER_REFLECT BORDER_ISOLATED
gaussblured_img = cv2.GaussianBlur(img, kernel_size, sigmaX, borderType) 

#cv2_imshow(gaussblured_img)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(gaussblured_img, cv2.COLOR_BGR2RGB))

## 1. b) Detekcja krawędzi

Detekcja krawędz może się odbywać na przykład przy użyciu odpowiediego filtra (maski), np:
* Laplacian - oblicza Laplacian obrazu

* Sobel - oparator ten obliczany jest w bibliotece OpenCV w jednym kierunku dlatego należy go poiczyć zarówno dla kierunku X jak i Y

* Canny - algorytm znajdywania krawędzi opracowany przez J. Canny w 1986 roku

Dla tych metod należy podać jako argument wejściowy ddepth/dtype (*Desired depth of the destination image*) czyli format obrazu wyjściowego np. uint8 lub float. Warto skorzystać tu z natywnych formatów OpenCV jak np. *cv2.CV_64F*.

In [ ]:
# Laplacian
ddepth = cv2.CV_64F # format obrazu wyjściowego
ksize = 3 # rozmiar filtra
img_laplacian = cv2.Laplacian(img,ddepth, ksize, borderType = cv2.BORDER_REPLICATE)

#cv2_imshow(img_laplacian)
#plt.figure(figsize=(10,10))
plt.imshow(img_laplacian, cmap='gray')

In [ ]:
# Sobel
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5) # parametr trzeci i czwarty określają kierunkowość, najpierw X, potem Y

frame_sobel = cv2.hconcat((sobelx, sobely))

#cv2_imshow(frame_sobel)
#plt.figure(figsize=(10,10))
plt.imshow(frame_sobel, cmap='gray')


In [ ]:
# Canny
# prócz obrazu wejściowego jako argumenty wejściowe podajemy dwa progi, gdzie wartości między tymi progami służą do wyznaczenia pikseli połaczonych następnie w krawędzie
threshold1 = 100
threshold2 = 200
img_canny = cv2.Canny(img,threshold1,threshold2)

#cv2_imshow(img_canny)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_canny, cv2.COLOR_BGR2RGB))

**!Uwaga!**
Ustawenia parametru ddepth/dtype ma znaczenie.

![alt text](https://github.com/knave88/APO/raw/main/double_edge.jpg)

W przypadku algorytmu Sobel'a i ostrych przejść czarne-białe i białe-czarne jeśli ustawimy dtype = cv2.CV_8U czyli uint8 to wykrywamy tylko jedną krawędz. Powód jest prosty. Wykryta kawędź ma wartości dodatnie natomiast pomijana ma wartości ujemne które nie mogą wystąpić w formacie uint8. Dlatego lepiej jest stosować "większy" format a nastepnie stosować wartość absolutną aby sprowadzić np. do uint8.

## 1. c) Wyostrzanie liniowe

Wyostrzanie obrazu podobnie jak wygładzanie może być zrealizowane przez zastosowanie filtracji i odpowiedniej maski. Użycie konwolucji w znacznym stopniu ułatwia zadanie. 
W bibliotece OpenCV filtrację dowolną maską można wykonać przy użyciu funkcji *filter2D*.

In [ ]:
# wyostrzanie
mask_sharp1 = np.array([[ 0,-1, 0],[-1, 5,-1],[ 0,-1, 0]])
mask_sharp2 = np.array([[-1,-1,-1],[-1, 9,-1],[-1,-1,-1]])
mask_sharp3 = np.array([[ 1,-2, 1],[-2, 5,-2],[ 1,-2, 1]])

mask_sharp = mask_sharp2
print('My mask:')
print(mask_sharp)
print('Result of filtering:')
img_sharp = cv2.filter2D(img,cv2.CV_64F, mask_sharp, borderType = cv2.BORDER_REPLICATE)

#cv2_imshow(img_sharp)
#plt.figure(figsize=(10,10))
plt.imshow(img_sharp, cmap='gray')


## 1. d) Kierunkowa detekcja krawędzi

Operator ten umożliwia wyznaczenie dla każdego piksela obrazu estymaty pochodnej kierunkowej w jednym z 8 kierunków. Przykładowe maski kierunkowe:

![alt text](https://github.com/knave88/APO/raw/main/Prewitt.png)

In [ ]:
# Prewitt dla kiernku NE
mask_prewittNE = np.array([[0,+1,+1],[-1,0,+1],[-1,-1,0]])
print('My mask:')
print(mask_prewittNE)
print('Result of filtering:')
img_prewitt = cv2.filter2D(img,cv2.CV_64F, mask_prewittNE, borderType = cv2.BORDER_REPLICATE)

#cv2_imshow(img_prewitt)
#plt.figure(figsize=(10,10))
plt.imshow(img_prewitt, cmap='gray')

## 1. e) Uniwersalna operacja liniowa sąsiedztwa

Uniwersalna operacja liniowa sąsiedztwa oparta na masce 3x3 o wartościach zadawanych w sposób interakcyjny przez użytkowanika. Należy zadbać o to aby program akceptował jadynie poprawny typ zmiennej oraz sprawdzał czy zadana wartość jest w określonym zakresie.

In [ ]:
from ipywidgets import GridspecLayout, BoundedIntText, Layout

def create_expanded_input(value):
    return BoundedIntText(description='', value=1, min=-100, layout=Layout(height='auto', width='auto'))

grid = GridspecLayout(3, 3)

for i in range(3):
    for j in range(3):
        grid[i, j] = create_expanded_input(0);
grid

In [ ]:
kernel = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        kernel[i, j] = int(grid[i, j].value)
        
print('My mask:')
kernel=np.int64(kernel)/np.sum(kernel)
print(kernel)
print('Result of filtering:')
img_filtered = cv2.filter2D(img,cv2.CV_64F, kernel, borderType = cv2.BORDER_REPLICATE)

#cv2_imshow(img_filtered)
#plt.figure(figsize=(10,10))
plt.imshow(img_filtered, cmap='gray')


# Filtr medianowy

Opracja filtracji medianowej również może być w prosty sposób zrealizowana przy użyciu biblioteki OpenCV.  Podobnie jak w przykładach powyżej, argumenty wejściowe, prócz obrazu wejściowego, to rozmiar filtra. 
**Uwaga!** Należy pamiętać, że rozmiar okna musi mieć nieprzystą wartość całkowitą większą od 1, np.: 3, 5, 7 itd.

Ponadto, w obrazach wielokanałowych każdy kanał jest przetwarzany odrębnie.

In [ ]:
medianBlured_img = cv2.medianBlur(img, 11) 

#cv2_imshow(medianBlured_img)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(medianBlured_img, cv2.COLOR_BGR2RGB))

# Operacje jednopunktowe dwuargumentowe

**Uwaga!** przy operacjach jednopunktowych dwuargumentowych należy zadbać aby oba obrazy miały ten sam rozmiar. Jeśli występuje niezgodność można zastosować jedną z dwóch opcji:
* resize = przeskalowanie
* padding = dopełnienie (np. wartością zero)

---

Aby program zadziałał należy dodać plik 'testing.bmp' (dostępny na UBI) tak samo jak obraz Leny.

In [ ]:
# wczytanie drugiego obrazu
img2 = cv2.imread('testing.bmp', cv2.IMREAD_GRAYSCALE)
# zmiana rozmiaru (dopasowanie) do obrazu Leny
img2 = cv2.resize(img2, img.shape) 

#cv2_imshow(img2)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

### 3.1 Dodawanie

In [ ]:
# dodawanie obrazów realizowane w bibliotece OpenCV
img_add = cv2.add(img, img2)

#cv2_imshow(img_add)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_add, cv2.COLOR_BGR2RGB))

### 3.2 Mieszanie (blending)

"Mieszanie" pozwala na doddanie do siebie obrazów z zadaną wagą dla każdego z nich. Pozwala to na uzyskanie efektu prześwitywania lub tworzenia tzw. 'overlay'. Dodatkowo pozwala również na dodanie zadanej wartości do każdej sumy (parametr gamma). Relizowane jest zgodnie z poniższym wzorem:

![alt text](https://github.com/knave88/APO/raw/main/addWeighted.png)



gdzie:
* alpha określa udział obrazu pierwszego
* beta określa udział obrazu drugiego
* gamma określa skalar dodany do każdego sumowanego piksela (wartość dodatnia lub ujemna)


In [ ]:
# przykład zastosowania
img_blend = cv2.addWeighted(img,0.7,img2,0.5,-100)

#cv2_imshow(img_blend)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_blend, cv2.COLOR_BGR2RGB))

### 3.3 Operacje logiczne

In [ ]:
# negacja = bitwise NOT
img2_inv = cv2.bitwise_not(img2)

frame3 = cv2.hconcat((img2, img2_inv))
#cv2_imshow(frame3)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(frame3, cv2.COLOR_BGR2RGB))

In [ ]:
# AND
img_and = cv2.bitwise_and(img, img2)
#cv2_imshow(img_and)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_and, cv2.COLOR_BGR2RGB))

In [ ]:
# OR
img_or = cv2.bitwise_or(img, img2)
#cv2_imshow(img_or)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_or, cv2.COLOR_BGR2RGB))

In [ ]:
# XOR
img_xor = cv2.bitwise_xor(img, img2)
#cv2_imshow(img_xor)
#plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img_xor, cv2.COLOR_BGR2RGB))

# Operacje sąsiedztwa – filtracja dwu i jedno etapowa

Przyjmijmy sytuację w której mamy jedną maskę filtracji *mF* o rozmiarze 3x3 oraz drugą maskę filtracji *mG* o rozmiarze 3x3. Obraz wejściowy w naszych założeniach możemy uznać za dwuwymiarowy sygnał wejściowy *x*. Wynik kolejnej filtracji (kaskadowej) dwoma maskami możemy zapisać w ten sposób:

`y=mG∗(mF∗x)`

Ważną cechą (i wiele ułatwiającą w tym wypadku) konwolucji jest jej łączność. Dzięki temu powyższy zapis można przedstawić również tak:

`y=g∗(mF∗x)=(mG∗mF)∗x=mH∗x`

Gdzie: `mH = mG*mF = mF*mG`

Z tego wynika, że możliwe jest uzyskanie jednej maski filtracji, dającej ten sam efekt, przez konwolucję dwóch masek podstawowych.



In [ ]:
# utworzenie pierwszej maski 3x3 - wygładzenie
mF = np.ones((3,3))
mF

In [ ]:
# utworzenie drugiej maski 3x3 - wyostrzenie
mG = np.array([[1, -2, 1], 
               [-2, 4, -2], 
               [1, -2, 1]])
mG

In [ ]:
# konstrukcja maski w oparciu o dwie powyższe maski 3x3 
# wykorzystanie konwolucji do wygenerowania maski 5x5
from scipy.signal import convolve2d as conv2 # funkcja konwolucji dwuwymiraowej
mH = conv2(mF, mG, mode='full') # mode full zapewnia odpowieni rozmiar maski
mH

In [ ]:
# wykonanie dwu etapowej filtracji z maskami 3x3 (jak w Lab3)
res_step1 = cv2.filter2D(img,cv2.CV_64F, mF, borderType = cv2.BORDER_REPLICATE)
res_step2 = cv2.filter2D(res_step1,cv2.CV_64F, mG, borderType = cv2.BORDER_REPLICATE)
#cv2_imshow(res_step22)

# wykonanie jednoetapowej filtracji z maską 5x5
res_5x5 = cv2.filter2D(img,cv2.CV_64F, mH, borderType = cv2.BORDER_REPLICATE)
#cv2_imshow(res_5x5)

# wizualne porównanie wyników
frame = cv2.hconcat((np.uint8(res_step2), np.uint8(res_5x5)))
#cv2_imshow(frame)
#plt.figure(figsize=(10,10))
plt.imshow(frame)

W przedstawiony powyżej sposób możemy przekształcić dowolne dwie maski w jedną. Należy jednak pamiętać o tym że maska wynikowa ma inny rozmiar niż maski bazowe.